# Web Scraping for Data Science Course

### Made by: Andrija Botica, Daria Milić, Karlo Nevešćanin
### Professor: dr. sc. Toni Perković

## Introduction

Our idea was to use web scraping to gather data from Croatian stores.

The data can then be used to find the product you want for the cheapest price.

Data will also be used in Human Computer Interaction course where we will implement full-stack web application.

## Getting Started

We used Python, BeautifulSoup and Selenium to scrape data from Croatian stores.

We scraped data from the following stores:
- Konzum - Webshop
- Ribola - Wolt
- Studenac - Wolt


To start scraping we need to install the following libraries:
- requests
- bs4
- lxml

In [ ]:
pip install requests bs4 lxml

Now we need to include them in our code (lxml does not need to be included).

In [2]:
import requests
from bs4 import BeautifulSoup

Konzum webshop has different categories of products, so we need to scrape them separately.

Every category has its own URL, so we need to scrape them one by one.

We define a variable `categories` which contains all the categories we want to scrape.
To get each category we will create `categories_links` and `categories_urls` lists.

The latter will have base URL appended to each category link.

In [3]:
# Konzum webshop
URL = "https://www.konzum.hr"

# Variables
categories = []
categories_links = []
categories_urls = []

Now let's scrape the data from Konzum webshop and extract the categories.

In [4]:
r = requests.get(URL)
soup = BeautifulSoup(r.content, "lxml")
table = soup.find("section", attrs={"class": "py-3"})
categories = table.find_all("a", attrs={"class": "category-box__link"})

We will create each categories URL using `for loops`.

In [5]:
for category in categories:
    categories_links.append(category['href'])

for links in categories_links:
    categories_urls.append(URL + links)

To keep count of the number of products we will use `total_products` variable.

In [6]:
total_products = 0

Now we can scrape the data from the categories.

For each category url from `categories_urls` we will scrape the data and extract the product name, price and image alongside with name of the store.

In [ ]:
for category_url in categories_urls:
            page = requests.get(f'{category_url}').text
            pageSoup = BeautifulSoup(page, 'lxml')
            subCategories = pageSoup.find('ul', class_='plain-list mb-3')
            subCategories_aTag = subCategories.find_all('a')

            if subCategories_aTag:
                for aTag in subCategories_aTag:
                    subCategoryLink = aTag.get('href')
                    subCategoryURL = URL + subCategoryLink

                    if subCategoryURL:
                        page_number = 1
                        while True:
                            finalPage = requests.get(f'{subCategoryURL}?page={page_number}')
                            if finalPage.status_code != 200:
                                break
                            finalSoup = BeautifulSoup(finalPage.text, 'lxml')
                            allItems = finalSoup.find('div', class_='col-12 col-md-12 col-lg-10')
                            if not allItems:
                                break

                            productsList = allItems.find('div', class_='product-list product-list--md-5 js-product-layout-container product-list--grid')
                            if not productsList:
                                break

                            articles = productsList.find_all('article', class_='product-item product-default')
                            if not articles:
                                break

                            for article in articles:
                                articleImageURL = article.find('img').get('src')
                                if articleImageURL:
                                    articleTittleTag = article.find('h4', class_='product-default__title')
                                    if articleTittleTag:
                                        articleNameTag = articleTittleTag.find('a', class_='link-to-product')
                                        if articleNameTag:
                                            articleName = articleNameTag.get_text(strip=True)
                                            if articleName:
                                                articleEuro = article.find('span', class_='price--kn').text
                                                articleCent = article.find('span', class_='price--li').text
                                                total_products += 1

                                                print(f'Name: {articleName}', f'Price: {articleEuro}.{articleCent}', f'Image URL: {articleImageURL}', f'Store: Konzum')
                            page_number += 1

print(f'Total number of products: {total_products}')

We have created this code by looking at HTML code of the website. We simply tell BeatifulSoup to find the tags that contain the data we need.